In [6]:
import numpy as np
import pandas as pd
from scipy import stats
import tqdm

def generate_data(group_size, base_rate, effect, other_brands_knowledge_rate=0.3):
    """
    Генерирует данные эксперимента.

    Parameters:
    -----------
    group_size : int
        Размер каждой группы (тест и контроль).
    base_rate : float
        Базовая доля знания бренда в контрольной группе (от 0 до 1).
    effect : float
        Относительный прирост знания в тестовой группе (например, 0.05 = +5%).
    other_brands_knowledge_rate : float
        Вероятность знания других брендов (не X).

    Returns:
    --------
    control_data, test_data : pandas.DataFrame
        Данные для контрольной и тестовой групп.
    """
    # Контрольная группа
    knows_x_control = np.random.binomial(1, base_rate, group_size)
    knows_other_control = np.random.binomial(1, other_brands_knowledge_rate, 
                                            (group_size, 3))  # 3 других бренда
    
    # Тестовая группа
    test_rate = base_rate * (1 + effect)
    test_rate = min(test_rate, 1.0)  # Не может превышать 100%
    knows_x_test = np.random.binomial(1, test_rate, group_size)
    knows_other_test = np.random.binomial(1, other_brands_knowledge_rate, 
                                         (group_size, 3))
    
    control_data = pd.DataFrame({
        'knows_x': knows_x_control,
        'knows_other1': knows_other_control[:, 0],
        'knows_other2': knows_other_control[:, 1],
        'knows_other3': knows_other_control[:, 2]
    })
    
    test_data = pd.DataFrame({
        'knows_x': knows_x_test,
        'knows_other1': knows_other_test[:, 0],
        'knows_other2': knows_other_test[:, 1],
        'knows_other3': knows_other_test[:, 2]
    })
    
    return control_data, test_data

# Пример генерации данных
control, test = generate_data(group_size=5000, base_rate=0.3, effect=0.1)
print(f"Контроль: знают бренд X - {control['knows_x'].mean():.3f}")
print(f"Тест: знают бренд X - {test['knows_x'].mean():.3f}")

Контроль: знают бренд X - 0.297
Тест: знают бренд X - 0.330


In [7]:
def z_test_proportions(control_data, test_data, alpha=0.05):
    """
    Z-тест для сравнения долей между двумя группами.
    
    Returns:
    --------
    significant : bool
        Статистически значимо ли отличие?
    p_value : float
        p-value теста
    effect_size : float
        Относительный прирост (test_rate / control_rate - 1)
    """
    n_control = len(control_data)
    n_test = len(test_data)
    
    p_control = control_data['knows_x'].mean()
    p_test = test_data['knows_x'].mean()
    
    # Объединенная доля
    p_pool = (control_data['knows_x'].sum() + test_data['knows_x'].sum()) / (n_control + n_test)
    
    # Стандартная ошибка
    se = np.sqrt(p_pool * (1 - p_pool) * (1/n_control + 1/n_test))
    
    # Z-статистика
    z = (p_test - p_control) / se
    
    # p-value (двусторонний тест)
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
    # Эффект
    effect_size = p_test / p_control - 1 if p_control > 0 else 0
    
    significant = p_value < alpha
    
    return significant, p_value, effect_size

In [8]:
def bootstrap_test(control_data, test_data, n_bootstrap=10000, alpha=0.05):
    """
    Bootstrap тест для сравнения долей.
    
    Returns:
    --------
    significant : bool
        Статистически значимо ли отличие?
    ci_lower, ci_upper : float
        Границы доверительного интервала для разницы долей
    effect_size : float
        Относительный прирост
    """
    control_values = control_data['knows_x'].values
    test_values = test_data['knows_x'].values
    
    bootstrap_diffs = []
    
    for _ in range(n_bootstrap):
        # Bootstrap выборки
        control_bootstrap = np.random.choice(control_values, size=len(control_values), replace=True)
        test_bootstrap = np.random.choice(test_values, size=len(test_values), replace=True)
        
        # Разница долей
        diff = test_bootstrap.mean() - control_bootstrap.mean()
        bootstrap_diffs.append(diff)
    
    # Доверительный интервал
    ci_lower = np.percentile(bootstrap_diffs, alpha/2 * 100)
    ci_upper = np.percentile(bootstrap_diffs, (1 - alpha/2) * 100)
    
    # Фактическая разница
    actual_diff = test_values.mean() - control_values.mean()
    effect_size = test_values.mean() / control_values.mean() - 1 if control_values.mean() > 0 else 0
    
    # Статистическая значимость: если 0 не входит в доверительный интервал
    significant = (ci_lower > 0) or (ci_upper < 0)
    
    return significant, (ci_lower, ci_upper), effect_size

In [9]:
def detect_effect_power_simulation(group_sizes, base_rates, effects, n_simulations=1000, alpha=0.05):
    """
    Моделирование мощности обнаружения эффектов при разных размерах выборки.
    
    Parameters:
    -----------
    group_sizes : list
        Список размеров групп для тестирования
    base_rates : list
        Список базовых уровней знания бренда
    effects : list
        Список относительных эффектов для тестирования
    n_simulations : int
        Количество симуляций для каждого сценария
    alpha : float
        Уровень значимости
    
    Returns:
    --------
    results : pandas.DataFrame
        Результаты симуляции
    """
    
    results = []
    
    for group_size in tqdm.tqdm(group_sizes, desc="Group Sizes"):
        for base_rate in base_rates:
            for effect in effects:
                # Проверяем, возможен ли такой эффект (не превышает 100%)
                if base_rate * (1 + effect) > 1:
                    continue
                
                detected_effects = []
                
                for sim in range(n_simulations):
                    # Генерируем данные
                    control, test = generate_data(group_size, base_rate, effect)
                    
                    # Проверяем значимость Z-тестом
                    significant, p_value, effect_measured = z_test_proportions(control, test, alpha)
                    
                    if significant:
                        detected_effects.append(effect_measured)
                
                # Если мы обнаружили эффект хотя бы в некоторых симуляциях
                if detected_effects:
                    median_effect = np.median(detected_effects)
                    q25_effect = np.percentile(detected_effects, 25)
                    q75_effect = np.percentile(detected_effects, 75)
                    detection_rate = len(detected_effects) / n_simulations
                else:
                    median_effect = q25_effect = q75_effect = np.nan
                    detection_rate = 0
                
                results.append({
                    'group_size': group_size,
                    'base_rate': base_rate,
                    'true_effect': effect,
                    'detection_rate': detection_rate,
                    'median_detected_effect': median_effect,
                    'q25_detected_effect': q25_effect,
                    'q75_detected_effect': q75_effect
                })
    
    return pd.DataFrame(results)

# Параметры для симуляции
group_sizes = [1000, 2000, 3000, 5000, 10000, 20000]
base_rates = [0.2, 0.5]  # Низкий и средний уровень знания бренда
effects = [0.01, 0.02, 0.03, 0.05, 0.08, 0.10, 0.15, 0.20]  # Разные размеры эффекта

results_df = detect_effect_power_simulation(
    group_sizes=group_sizes,
    base_rates=base_rates, 
    effects=effects,
    n_simulations=500 
)

Group Sizes: 100%|██████████| 6/6 [00:50<00:00,  8.49s/it]


In [10]:
def analyze_detection_thresholds(results_df, detection_threshold=0.8):
    """
    Анализирует, какие эффекты можно обнаружить с заданной мощностью.
    
    Parameters:
    -----------
    results_df : pandas.DataFrame
        Результаты симуляции
    detection_threshold : float
        Минимальная частота обнаружения для считания эффекта обнаруживаемым
    
    Returns:
    --------
    threshold_table : pandas.DataFrame
        Таблица с порогами обнаружения
    """
    
    threshold_data = []
    
    for group_size in group_sizes:
        for base_rate in base_rates:
            # Фильтруем данные для текущего сценария
            scenario_data = results_df[
                (results_df['group_size'] == group_size) & 
                (results_df['base_rate'] == base_rate)
            ].sort_values('true_effect')
            
            # Находим минимальный эффект, который обнаруживается с заданной мощностью
            detectable_effects = scenario_data[scenario_data['detection_rate'] >= detection_threshold]
            
            if not detectable_effects.empty:
                min_detectable_effect = detectable_effects['true_effect'].min()
                corresponding_stats = detectable_effects[detectable_effects['true_effect'] == min_detectable_effect].iloc[0]
                
                threshold_data.append({
                    'group_size': group_size,
                    'base_rate': base_rate,
                    'min_detectable_effect': min_detectable_effect,
                    'detection_rate': corresponding_stats['detection_rate'],
                    'median_effect': corresponding_stats['median_detected_effect'],
                    'q25_effect': corresponding_stats['q25_detected_effect'],
                    'q75_effect': corresponding_stats['q75_detected_effect']
                })
    
    return pd.DataFrame(threshold_data)

# Создаем таблицу порогов обнаружения
threshold_table = analyze_detection_thresholds(results_df, detection_threshold=0.8)
print("Пороги обнаружения эффектов (мощность ≥ 80%):")
print(threshold_table.to_string(index=False))

# Cоздаем сводную таблицу по разным размерам групп
pivot_table = results_df.pivot_table(
    index=['group_size', 'base_rate'], 
    columns='true_effect', 
    values='detection_rate'
)
print("\nМатрица мощности обнаружения:")
print(pivot_table.round(3))

Пороги обнаружения эффектов (мощность ≥ 80%):
 group_size  base_rate  min_detectable_effect  detection_rate  median_effect  q25_effect  q75_effect
       1000        0.5                   0.15           0.904       0.155030    0.127264    0.184891
       2000        0.2                   0.20           0.866       0.207692    0.168646    0.255319
       2000        0.5                   0.10           0.872       0.103688    0.083504    0.123542
       3000        0.2                   0.15           0.816       0.163807    0.137247    0.200169
       3000        0.5                   0.08           0.856       0.084193    0.069779    0.100116
       5000        0.2                   0.15           0.940       0.148790    0.121662    0.178491
       5000        0.5                   0.08           0.980       0.080749    0.066306    0.094750
      10000        0.2                   0.08           0.800       0.088206    0.075477    0.107133
      10000        0.5                   0.05

In [ ]:
"""
Выводы:
Классический Z-тест хорошо подходит для сравнения долей и быстрее вычисляется.

Bootstrap метод более устойчив к предположениям о распределении и дает наглядные доверительные интервалы.

Обнаруживаемость эффектов зависит от:

- Размера выборки (больше выборка = меньшие эффекты можно обнаружить)

- Базового уровня знания (при низком базовом уровне сложнее обнаружить маленькие относительные приросты)

- Величины эффекта


При 5000 анкет в группе можно надежно обнаруживать эффекты от 5-8%

Для обнаружения эффектов 1-3% нужны выборки от 20000 анкет

При низком базовом уровне знания (<20%) обнаруживать эффекты сложнее

"""